# Lab 10.2: Fertilization Strategy Analyzer
## Chapter 10: The Reproduction Revolution

### 🎯 Learning Objectives
- Compare external vs internal fertilization efficiency
- Calculate fertilization success under different conditions
- Analyze environmental effects on fertilization
- Model sperm competition and synchronization
- Predict optimal strategy for environments

### 📖 Connection to Chapter 10
This lab integrates **Section 10.2: Fertilization Strategies**:
- External fertilization (broadcast spawning)
- Internal fertilization (copulation)
- Coral spawning synchronization
- Shark reproductive diversity
- Stickleback nest guarding
- Trade-offs and evolution

### 🐟 The Question
**Why do salmon release millions of eggs in water while mammals have internal fertilization?**  
What determines which strategy works? Let's calculate!

In [ ]:
# === GOOGLE COLAB SETUP ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import *
from IPython.display import display, clear_output
from datetime import datetime

print("✓ Ready!")

## Part 1: Fertilization Theory

### Two Fundamental Strategies

**External Fertilization**:
- Eggs and sperm released into water
- Requires aquatic environment
- Massive gamete numbers
- Low individual egg survival

**Internal Fertilization**:
- Sperm delivered directly to female
- Works in any environment
- Fewer gametes needed
- Higher egg survival

### External Fertilization Success

$$F_{ext} = N_{eggs} \times P_{encounter} \times P_{sync} \times P_{survive}$$

Where:
- N_eggs = number of eggs released
- P_encounter = probability sperm meets egg
- P_sync = synchronization with males
- P_survive = egg survival (predation, environment)

### Encounter Probability

$$P_{encounter} = \frac{sperm\_density \times volume}{dilution\_factor}$$

Affected by:
- Water flow (stronger → lower)
- Distance between mates
- Sperm density
- Timing synchronization

### Internal Fertilization Success

$$F_{int} = N_{eggs} \times P_{mate} \times P_{transfer} \times P_{survive}$$

Where:
- N_eggs = eggs produced (fewer!)
- P_mate = finding/attracting mate
- P_transfer = successful copulation
- P_survive = internal development survival (high!)

### The Trade-off

**External**:
- ✅ Millions of eggs possible
- ✅ No complex anatomy needed
- ❌ Low fertilization rate (~1-10%)
- ❌ Requires water

**Internal**:
- ✅ High fertilization rate (>90%)
- ✅ Works on land
- ✅ Parental care easier
- ❌ Complex anatomy
- ❌ Fewer eggs

### From Chapter 10
**Examples**:
- Coral: Mass spawning, perfect synchronization
- Salmon: Migrate to spawning grounds
- Frogs: Amplexus (male holds female)
- Sharks: THREE strategies! (eggs, ovoviviparous, viviparous)
- Mammals: Internal + placenta

In [ ]:
# Functions
def calc_external_fertilization(n_eggs, sperm_density, water_flow, sync_quality, survival):
    """
    Calculate external fertilization success
    
    sperm_density: 0-1 (0=low, 1=high)
    water_flow: 0-1 (0=still, 1=strong current)
    sync_quality: 0-1 (0=poor timing, 1=perfect sync)
    survival: 0-1 (environmental survival)
    """
    # Encounter decreases with flow
    p_encounter = sperm_density * (1 - 0.7 * water_flow)
    
    # Total success
    fertilized = n_eggs * p_encounter * sync_quality
    survivors = fertilized * survival
    
    return fertilized, survivors

def calc_internal_fertilization(n_eggs, mate_success, transfer_success, survival):
    """
    Calculate internal fertilization success
    
    mate_success: 0-1 (finding mate)
    transfer_success: 0-1 (copulation success)
    survival: 0-1 (development survival)
    """
    fertilized = n_eggs * mate_success * transfer_success
    survivors = fertilized * survival
    
    return fertilized, survivors

# Test
print("EXAMPLES:")
print("="*60)

# Salmon (external)
n = 4000
fert, surv = calc_external_fertilization(
    n_eggs=n, sperm_density=0.6, water_flow=0.3,
    sync_quality=0.8, survival=0.001
)
print(f"SALMON (external):")
print(f"  Eggs: {n:,}")
print(f"  Fertilized: {fert:.0f}")
print(f"  Survivors: {surv:.0f}\n")

# Mammal (internal)
n = 2
fert, surv = calc_internal_fertilization(
    n_eggs=n, mate_success=0.8, transfer_success=0.95,
    survival=0.85
)
print(f"MAMMAL (internal):")
print(f"  Eggs: {n}")
print(f"  Fertilized: {fert:.1f}")
print(f"  Survivors: {surv:.1f}")

print("\n✓ Ready!")

## Part 2: Species Database

In [ ]:
species_fertilization = {
    'Coral': {
        'strategy': 'External', 'n_eggs': 1000000,
        'sperm_density': 0.9, 'water_flow': 0.2, 'sync': 0.95,
        'survival': 0.0001, 'type': 'Broadcast spawning',
        'desc': 'Mass synchronization! Entire reef spawns same night. '
                'Moon phase triggers. Overwhelms predators.'
    },
    'Salmon': {
        'strategy': 'External', 'n_eggs': 4000,
        'sperm_density': 0.6, 'water_flow': 0.3, 'sync': 0.8,
        'survival': 0.001, 'type': 'Nest spawning',
        'desc': 'Female digs nest in gravel. Male fertilizes. '
                'Buries eggs. Semelparous (breed once, die).'
    },
    'Frog': {
        'strategy': 'External', 'n_eggs': 2000,
        'sperm_density': 0.7, 'water_flow': 0.1, 'sync': 0.9,
        'survival': 0.01, 'type': 'Amplexus',
        'desc': 'Male grasps female (amplexus). Releases sperm '
                'as female lays eggs. High synchronization!'
    },
    'Stickleback': {
        'strategy': 'External', 'n_eggs': 200,
        'sperm_density': 0.8, 'water_flow': 0.2, 'sync': 0.95,
        'survival': 0.15, 'type': 'Nest with care',
        'desc': 'Male builds nest. Female lays inside. Male '
                'fertilizes and guards. High survival!'
    },
    'Shark (Oviparous)': {
        'strategy': 'Internal', 'n_eggs': 20,
        'mate_success': 0.7, 'transfer': 0.9, 'survival': 0.3,
        'type': 'Egg cases',
        'desc': 'Internal fertilization. Lays egg cases '
                '("mermaid purses"). External development.'
    },
    'Shark (Ovoviviparous)': {
        'strategy': 'Internal', 'n_eggs': 10,
        'mate_success': 0.7, 'transfer': 0.9, 'survival': 0.6,
        'type': 'Live birth (eggs)',
        'desc': 'Internal fertilization and development. '
                'Eggs hatch inside mother. Gives birth to live young!'
    },
    'Guppy': {
        'strategy': 'Internal', 'n_eggs': 30,
        'mate_success': 0.9, 'transfer': 0.95, 'survival': 0.5,
        'type': 'Viviparity',
        'desc': 'Internal fertilization and development. '
                'True live birth. Females store sperm!'
    },
    'Reptile': {
        'strategy': 'Internal', 'n_eggs': 12,
        'mate_success': 0.6, 'transfer': 0.85, 'survival': 0.25,
        'type': 'Amniotic egg',
        'desc': 'Internal fertilization. Lays shelled eggs '
                'on land. Amniotic membrane = terrestrial!'
    },
    'Bird': {
        'strategy': 'Internal', 'n_eggs': 4,
        'mate_success': 0.8, 'transfer': 0.90, 'survival': 0.60,
        'type': 'Amniotic + care',
        'desc': 'Internal fertilization. Hard-shelled eggs. '
                'Parental care boosts survival!'
    },
    'Mammal': {
        'strategy': 'Internal', 'n_eggs': 2,
        'mate_success': 0.8, 'transfer': 0.95, 'survival': 0.85,
        'type': 'Placental',
        'desc': 'Internal fertilization and development. '
                'Placenta nourishes fetus. Highest survival!'
    }
}

# Calculate for all
for name, sp in species_fertilization.items():
    if sp['strategy'] == 'External':
        fert, surv = calc_external_fertilization(
            sp['n_eggs'], sp['sperm_density'], sp['water_flow'],
            sp['sync'], sp['survival']
        )
    else:
        fert, surv = calc_internal_fertilization(
            sp['n_eggs'], sp['mate_success'], sp['transfer'],
            sp['survival']
        )
    sp['fertilized'] = fert
    sp['survivors'] = surv

print("FERTILIZATION STRATEGY DATABASE")
print("="*80)
print(f"{'Species':<25}{'Strategy':<12}{'Eggs':<12}{'Survivors'}")
print("="*80)
for name in sorted(species_fertilization.keys(), 
                  key=lambda x: species_fertilization[x]['n_eggs'], reverse=True):
    sp = species_fertilization[name]
    print(f"{name:<25}{sp['strategy']:<12}{sp['n_eggs']:<12,}{sp['survivors']:.1f}")
print("="*80)
print("✓ Database ready!")

## Part 3: Strategy Simulator

In [ ]:
def simulate_fertilization(species_name):
    sp = species_fertilization[species_name]
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Fertilization Pathway', 'Success Rates')
    )
    
    # 1. Pathway funnel
    if sp['strategy'] == 'External':
        stages = ['Eggs Released', 'Encounter Sperm', 'Fertilized', 'Survive']
        values = [
            sp['n_eggs'],
            sp['n_eggs'] * sp['sperm_density'] * (1 - 0.7*sp['water_flow']),
            sp['fertilized'],
            sp['survivors']
        ]
    else:
        stages = ['Eggs Produced', 'Mating Success', 'Fertilized', 'Survive']
        values = [
            sp['n_eggs'],
            sp['n_eggs'] * sp['mate_success'],
            sp['fertilized'],
            sp['survivors']
        ]
    
    fig.add_trace(go.Funnel(
        y=stages, x=values,
        textposition='inside',
        textinfo='value',
        marker={'color': ['#3498DB', '#2ECC71', '#F39C12', '#E74C3C']}
    ), row=1, col=1)
    
    # 2. Success rates
    metrics = ['Fertilization\nRate', 'Survival\nRate', 'Overall\nSuccess']
    rates = [
        100 * sp['fertilized'] / sp['n_eggs'],
        100 * sp['survivors'] / sp['fertilized'] if sp['fertilized'] > 0 else 0,
        100 * sp['survivors'] / sp['n_eggs']
    ]
    
    fig.add_trace(go.Bar(
        x=metrics, y=rates,
        marker_color=['#2ECC71', '#F39C12', '#3498DB'],
        text=[f"{r:.1f}%" for r in rates],
        textposition='outside'
    ), row=1, col=2)
    
    fig.update_yaxes(title_text="%", row=1, col=2)
    fig.update_layout(height=500, title_text=f"<b>{species_name}: {sp['type']}</b>")
    
    # Summary
    print("\n" + "="*70)
    print(f"{species_name.upper()}: {sp['strategy']} Fertilization")
    print("="*70)
    print(f"Type: {sp['type']}")
    print(f"\nNUMBERS:")
    print(f"  Eggs/gametes: {sp['n_eggs']:,}")
    print(f"  Fertilized: {sp['fertilized']:.0f} ({100*sp['fertilized']/sp['n_eggs']:.1f}%)")
    print(f"  Survivors: {sp['survivors']:.1f} ({100*sp['survivors']/sp['n_eggs']:.2f}%)")
    print(f"\n{sp['desc']}")
    print("="*70)
    
    fig.show()

species_dropdown = Dropdown(
    options=sorted(species_fertilization.keys()),
    value='Salmon',
    description='Species:'
)

display(HTML("<h3>🐟 Fertilization Simulator</h3>"))
interact(simulate_fertilization, species_name=species_dropdown);

## Part 4: Environmental Effects

In [ ]:
def environmental_comparison(water_flow_level):
    """Compare external fertilization under different water flow"""
    
    # Test species: Salmon
    n_eggs = 4000
    sperm_density = 0.6
    sync = 0.8
    survival = 0.001
    
    flows = np.linspace(0, 1, 50)
    survivors = []
    
    for flow in flows:
        _, surv = calc_external_fertilization(
            n_eggs, sperm_density, flow, sync, survival
        )
        survivors.append(surv)
    
    # Current condition
    _, current_surv = calc_external_fertilization(
        n_eggs, sperm_density, water_flow_level, sync, survival
    )
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=flows, y=survivors,
        mode='lines', line=dict(color='#3498DB', width=3),
        fill='tozeroy', name='Survivors'
    ))
    fig.add_vline(
        x=water_flow_level, line_dash='dash',
        annotation_text=f"Current: {current_surv:.0f} survivors"
    )
    
    fig.update_layout(
        height=400,
        title='Effect of Water Flow on External Fertilization',
        xaxis_title='Water Flow (0=still, 1=strong current)',
        yaxis_title='Survivors'
    )
    
    print(f"\nAt flow level {water_flow_level:.2f}:")
    print(f"  Survivors: {current_surv:.0f}")
    print(f"  Reduction: {100*(1-current_surv/survivors[0]):.1f}% vs still water")
    
    fig.show()

flow_slider = FloatSlider(
    value=0.3, min=0, max=1, step=0.05,
    description='Water Flow:',
    style={'description_width': '100px'}
)

display(HTML("<h3>🌊 Environmental Effects</h3>"))
interact(environmental_comparison, water_flow_level=flow_slider);

## Part 5: Challenges

### Challenge 1: Synchronization Timing ⏰

**Question**: How long is the optimal spawning window?

**Scenario**: Coral spawning
- 1 million eggs per colony
- Sperm viability: 2 hours in water
- Current disperses gametes 100m/hour

**If too early**: Eggs released before sperm arrive
**If too late**: Sperm already diluted/dead
**Window**: ±1 hour?

<details>
<summary>Solution</summary>

**Sperm concentration over time**:
- t=0: Peak density (1.0)
- t=1hr: 50% viable, dispersed
- t=2hr: 10% viable, highly dispersed

**Optimal window**: ±30 minutes
- Earlier: Only 30% sperm available
- Later: Sperm dying/dispersed

**Real corals**: Sync within 15-30 minutes!
- Moon phase trigger
- Temperature cue
- Chemical signals

**Insight**: Tight synchronization critical for external fertilization!
</details>

### Challenge 2: Sperm Competition 🏊

**Question**: Multiple males fertilizing same eggs - who wins?

**Scenario**: Female frog, 2000 eggs
- Male A: 1 million sperm, arrives first
- Male B: 5 million sperm, arrives 10 sec later

**Model**: First male advantage? Or numbers win?

<details>
<summary>Solution</summary>

**Fertilization is fast**: ~seconds per egg

**Scenario 1** (slow fertilization, 1 min each):
- Male A fertilizes ~30 eggs before Male B arrives
- Remaining 1970 eggs: sperm ratio 1:5
- Male A: 30 + (1970 × 0.17) = ~365 eggs
- Male B: 1970 × 0.83 = ~1635 eggs
- **Numbers win!**

**Scenario 2** (fast fertilization, 1 sec each):
- Male A fertilizes ~10 eggs before Male B
- But most eggs still available
- Similar outcome: Male B wins

**Real frogs**: Males compete by:
- Getting to female first (amplexus)
- Producing more sperm
- Staying longer

**Evolution**: Arms race for sperm production!
</details>

### Challenge 3: Evolution of Internal Fertilization 🦎

**Question**: When is internal worth the anatomical cost?

**Trade-off calculation**:
- External: 4000 eggs, 1% fertilized, 0.1% survive = 4 survivors
- Internal: 40 eggs (100× fewer), 90% fertilized, 25% survive = ?

**Consider**: Cost of copulatory organs, mating behavior, gestation

<details>
<summary>Solution</summary>

**External**: 4000 × 0.01 × 0.001 = 4 survivors

**Internal**: 40 × 0.90 × 0.25 = 9 survivors

**Internal wins IF**:
- Can increase offspring survival >2.5×
- Reduces egg number <100×
- Anatomical cost is justified

**When internal evolves**:
✅ Terrestrial environments (no water for external)
✅ Unpredictable aquatic environments
✅ High predation on eggs
✅ Parental care possible

**Examples**:
- Sharks: All three strategies! (flexibility)
- Reptiles: Internal + amniotic egg = land
- Mammals: Internal + placenta = max survival

**Pattern**: Internal fertilization = gateway to terrestrial life!
</details>

## Part 6: Export

In [ ]:
def export_results():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"
    
    data = []
    for name, sp in species_fertilization.items():
        data.append({
            'Species': name, 'Strategy': sp['strategy'],
            'Eggs': sp['n_eggs'], 'Type': sp['type'],
            'Fertilized': sp['fertilized'],
            'Survivors': sp['survivors']
        })
    
    df = pd.DataFrame(data)
    csv_file = f"{output_dir}/lab_10_4_fertilization_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved: {csv_file}")
    print("\nDownload: 📁 → /content → right-click")

btn = Button(description='📥 Export', button_style='success', icon='download')
btn.on_click(lambda b: export_results())
display(HTML("<h3>📤 Export</h3>"))
display(btn)

## Summary

### Key Insights

✅ **Two strategies** - External vs internal fertilization  
✅ **Trade-offs** - Quantity vs quality, environment dependency  
✅ **Synchronization** - Critical for external success  
✅ **Environment shapes strategy** - Water/land, flow, predation  
✅ **Evolution** - Internal = gateway to terrestrial life  

### Strategy Comparison

**External Wins When**:
- Aquatic environment available
- Can produce millions of gametes
- Synchronization possible
- Examples: Coral, Salmon, Frogs

**Internal Wins When**:
- Terrestrial environment
- Parental care valuable
- Quality over quantity
- Examples: Reptiles, Birds, Mammals

### Amazing Facts

- Coral: Entire reef synchronizes to 30 minutes!
- Salmon: Return to natal stream to spawn
- Sharks: ALL THREE strategies in one group!
- Stickleback: Male builds nest, guards eggs
- Mammals: 99%+ fertilization success

### The Pattern

**Environment determines strategy**:
- Water → External possible
- Land → Internal required
- High predation → Internal better
- Parental care → Internal synergy

**Evolution**: Internal fertilization enabled vertebrate terrestrial invasion!

**Congratulations!** 🎉